In [5]:
import pandas as pd
import yfinance as yf

# Import financial data for stocks
tickers = ['AAPL', 'MSFT', 'TSLA', 'GOOGL']
data = yf.download(tickers, start="2020-01-01", end="2023-01-01")['Adj Close']

# Fetch ESG data from Yahoo Finance
esg_scores = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    try:
        # Fetch the total ESG score if available
        esg_scores[ticker] = stock.sustainability.loc['totalEsgScore'].values[0]
    except:
        # Handle cases where ESG data isn't available
        esg_scores[ticker] = None

# Convert to DataFrame
esg_data = pd.DataFrame(list(esg_scores.items()), columns=['ticker', 'esg_score'])

# Remove rows with missing ESG scores
esg_data.dropna(inplace=True)

# Calculate daily returns
returns = data.pct_change()

# Calculate annualized volatility (risk)
annualized_volatility = returns.std() * (252 ** 0.5)

# Construct decarbonized portfolio (ESG ranking)
esg_data['esg_rank'] = esg_data['esg_score'].rank()
low_carbon_stocks = esg_data[esg_data['esg_rank'] < esg_data['esg_rank'].median()]
low_carbon_portfolio_tickers = low_carbon_stocks['ticker'].tolist()

# Filter the financial data to only include stocks with ESG scores
filtered_data = data[low_carbon_portfolio_tickers]

# Calculate portfolio returns and Sharpe ratio
if not filtered_data.empty:
    portfolio_returns = filtered_data.pct_change().mean() * 252
    portfolio_volatility = filtered_data.pct_change().std() * (252 ** 0.5)

    # Calculate Sharpe ratio
    risk_free_rate = 0.01  # Example risk-free rate
    sharpe_ratio = (portfolio_returns - risk_free_rate) / portfolio_volatility
    print("Portfolio Sharpe Ratio:", sharpe_ratio)
else:
    print("No valid data for portfolio returns.")

# Handle missing ESG scores
if not esg_data['esg_score'].isna().all():
    portfolio_esg_score = esg_data[esg_data['ticker'].isin(low_carbon_portfolio_tickers)]['esg_score'].mean()
    print("Portfolio ESG Score:", portfolio_esg_score)
else:
    print("No valid ESG scores for portfolio.")


[*********************100%***********************]  4 of 4 completed


No valid data for portfolio returns.
No valid ESG scores for portfolio.
